In [117]:
import json
import urllib
import pandas as pd
import numpy as np
import requests
import json
import re
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import random
import time
%matplotlib inline

We import our scraped dataset from the API lab, which has information on the movies in IMDB's top 250 movies list.

In [178]:
df = pd.read_csv('scrapetest.csv')
df.head(1)

,Unnamed: 0,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,...,Released,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes
0,0,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Nominated for 7 Oscars. Another 19 wins & 30 n...,USA,Frank Darabont,"Crime, Drama",English,80.0,Two imprisoned men bond over a number of years...,https://images-na.ssl-images-amazon.com/images...,...,14 Oct 1994,True,142 min,The Shawshank Redemption,movie,"Stephen King (short story ""Rita Hayworth and S...",1994,tt0111161,9.3,"1,786,262"


We will add a new column that indicates that all these 250 movies are part of the list.

In [179]:
df['top250'] = 1

In [180]:
df.head(1)

,Unnamed: 0,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,...,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes,top250
0,0,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Nominated for 7 Oscars. Another 19 wins & 30 n...,USA,Frank Darabont,"Crime, Drama",English,80.0,Two imprisoned men bond over a number of years...,https://images-na.ssl-images-amazon.com/images...,...,True,142 min,The Shawshank Redemption,movie,"Stephen King (short story ""Rita Hayworth and S...",1994,tt0111161,9.3,"1,786,262",1


In [181]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

We make a list of the imdbIDs so we can check if any of our new scraped data is already in the top 250 list.

In [6]:
id250 = list(df['imdbID'])

We will scrape data from various IMDB pages to acquire data from movies that aren't in the top 250 list.

We will aim for at least 250 new movies

In [25]:
#checker
checker = 250

Lets get some movies that are currently in theaters

In [7]:
top_movies = requests.get('http://www.imdb.com/chart/boxoffice?ref_=nv_ch_cht_1')

with open('top_boxoffice.html', 'w') as f:
    f.write(top_movies.content)



In [10]:
with open('top_boxoffice.html', 'r') as f:
    content = BeautifulSoup(f.read())

In [16]:
main_table = content.find('tbody') #<tdbody>

In [17]:
movie_ids = []

for movie in main_table.find_all('td', {'class': 'watchlistColumn'}):
    for div_tag in movie.find_all('div'):
        try:
            movie_ids.append(div_tag['data-tconst'])
        except:
            pass

In [19]:
movie_rows = []
for movie in movie_ids:
    r = requests.get('http://www.omdbapi.com/?i=%s&r=json' % movie)
    json_content = json.loads(r.content)
    movie_rows.append(json_content)

In [39]:
dfbox = pd.DataFrame(movie_rows)
dfbox.shape
dfbox.to_csv('dfbox.csv', encoding = 'utf-8')
checker = checker - dfbox.shape[0]

In [27]:
checker

240

240 more movies to go, lets randomly sample movies by year. We want a diverse range of movies, so we will scrape by genres. However, IMDB only sorts movies by genre by popularity. We will scrape as many movies as possible and hopefully have a large enough sample size to randomly select movies to fill up our dataset.

Here we scrape 100 of the worst movies on IMDB

In [40]:
bot_movies = requests.get('http://www.imdb.com/chart/bottom')

with open('bottom.html', 'w') as f:
    f.write(bot_movies.content)


In [41]:
with open('bottom.html', 'r') as f:
    content = BeautifulSoup(f.read())

In [42]:
main_table = content.find('tbody', {'class': 'lister-list'}) #<tbody class='lister-list'>

In [44]:
movie_ids = []

for movie in main_table.find_all('td', {'class': 'ratingColumn'}):
    for div_tag in movie.find_all('div'):
        try:
            movie_ids.append(div_tag['data-titleid'])
        except:
            pass

In [45]:
movie_rows = []
for movie in movie_ids:
    r = requests.get('http://www.omdbapi.com/?i=%s&r=json' % movie)
    json_content = json.loads(r.text)
    movie_rows.append(json_content)

In [47]:
dfbot = pd.DataFrame(movie_rows)
dfbot.to_csv('dfbot.csv', encoding = 'utf-8')

Since I need as many movies as possible, to save time, I downloaded a dataset of 5000 IMDB movies scraped around 7 months ago that was uploaded to Kaggle. 

In [49]:
df5000 = pd.read_csv('movie_metadata.csv')
df5000.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


Since the dataset has features that are different than our original dataset, we only want to extract the imdbID, and feed it to OMDBAPI again. To do this, we have to split the movie_imdb_link column values.

In [55]:
df5000['movie_imdb_link'][0].split('/')[4]
#this gives us the imdbID

'tt0499549'

In [88]:
lis5000 = [x.split('/')[4] for x in df5000['movie_imdb_link']]

However, it might be better to add the IMDBid's of our previous scraped datasets as well and sample from one list

In [68]:
box = [str(x) for x in dfbox['imdbID']]
bot = [str(x) for x in dfbot['imdbID']]

In [90]:
lis5000.extend(box)

In [91]:
lis5000.extend(bot)

In [92]:
len(lis5000)

5153

We don't want to imbalance our dataset with 250 1s and 5153 0s, so we will only sample 250 movies from the list. 

In [114]:
samplelis = [x for x in lis5000 if x not in id250]
len(samplelis)

4988

In [115]:
sampleid = random.sample(samplelis, 250)

In [142]:
movie_rows = []
for movie in sampleid[0:200]:
    r = requests.get('http://www.omdbapi.com/?i=%s&r=json' % movie)
    json_content = json.loads(r.content)
    movie_rows.append(json_content)
    time.sleep(1)

In [143]:
dfsample1 = pd.DataFrame(movie_rows)
dfsample1.to_csv('dfsample1.csv', encoding = 'utf-8')

In [145]:
movie_rows = []
for movie in sampleid[200:251]:
    r = requests.get('http://www.omdbapi.com/?i=%s&r=json' % movie)
    json_content = json.loads(r.content)
    movie_rows.append(json_content)
    time.sleep(2)

In [146]:
dfsample2 = pd.DataFrame(movie_rows)
dfsample2.to_csv('dfsample2.csv', encoding = 'utf-8')

I just had to separate the movie IDs as I was rate-limited when I tried requesting all 250 at once.

In [147]:
listofdf = [dfsample1, dfsample2]
dftomerge = pd.concat(listofdf)
dftomerge.shape

(250, 21)

In [148]:
dftomerge.reset_index(drop = True)

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,...,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes,totalSeasons
0,"Adam Beach, Evan Adams, Irene Bedard, Gary Farmer",11 wins & 8 nominations.,"Canada, USA",Chris Eyre,"Comedy, Drama",English,76,Young Indian man Thomas is a nerd in his reser...,https://images-na.ssl-images-amazon.com/images...,PG-13,...,True,89 min,Smoke Signals,movie,"Sherman Alexie (book), Sherman Alexie (screenp...",1998,tt0120321,7.2,"8,416",NaN
1,"Robin Williams, Mandy Moore, John Krasinski, E...",1 nomination.,"USA, Australia",Ken Kwapis,"Comedy, Romance",English,25,A reverend puts an engaged couple through a gr...,https://images-na.ssl-images-amazon.com/images...,PG-13,...,True,91 min,License to Wed,movie,"Kim Barker (screenplay), Tim Rasmussen (screen...",2007,tt0762114,5.3,"33,017",NaN
2,"Victor Rasuk, Donna Maldonado, Kevin Rivera, K...",4 wins & 12 nominations.,"France, USA",Peter Sollett,"Drama, Romance","English, Spanish",83,A Lower East Side teen-ager struggles to find ...,https://images-na.ssl-images-amazon.com/images...,R,...,True,88 min,Raising Victor Vargas,movie,"Peter Sollett (story), Peter Sollett, Eva Vive...",2002,tt0316188,7.2,"4,811",NaN
3,"Russell Crowe, Elizabeth Banks, Michael Buie, ...",4 nominations.,"USA, France",Paul Haggis,"Crime, Drama, Romance",English,52,A married couple's life is turned upside down ...,https://images-na.ssl-images-amazon.com/images...,PG-13,...,True,133 min,The Next Three Days,movie,"Paul Haggis (screenplay), Fred Cavayé (screenp...",2010,tt1458175,7.4,"151,013",NaN
4,"Dane Cook, Kate Hudson, Alec Baldwin, Jason Biggs",1 nomination.,USA,Howard Deutch,"Comedy, Romance",English,34,Tank faces the ultimate test of friendship whe...,https://images-na.ssl-images-amazon.com/images...,R,...,True,101 min,My Best Friend's Girl,movie,Jordan Cahan,2008,tt1046163,5.9,"37,990",NaN
5,"Mike Starr, Paul Herman, Don Pardo, Martin Ros...",Nominated for 2 Oscars. Another 3 wins & 7 nom...,USA,Woody Allen,Comedy,"English, Spanish, Italian",N/A,A nostalgic look at radio's golden age focusin...,https://images-na.ssl-images-amazon.com/images...,PG,...,True,88 min,Radio Days,movie,Woody Allen,1987,tt0093818,7.6,"25,475",NaN
6,"Nicole Kidman, Joseph Fiennes, Hugo Weaving, L...",1 win & 6 nominations.,"Ireland, Australia",Kim Farrant,"Drama, Mystery, Thriller",English,42,A family finds their dull life in a rural outb...,https://images-na.ssl-images-amazon.com/images...,R,...,True,112 min,Strangerland,movie,"Michael Kinirons, Fiona Seres (story), Fiona S...",2015,tt2325977,5.2,"6,298",NaN
7,"Jordana Brewster, Taylor Handley, Diora Baird,...",5 nominations.,USA,Jonathan Liebesman,Horror,English,30,On one last road trip before they're sent to s...,https://images-na.ssl-images-amazon.com/images...,R,...,True,91 min,The Texas Chainsaw Massacre: The Beginning,movie,"Sheldon Turner (screenplay), Sheldon Turner (s...",2006,tt0420294,5.9,"57,990",NaN
8,"Siem Vroom, Marlies van Alcmaer, Erik van 't W...",Won 3 BAFTA Film Awards. Another 4 wins & 4 no...,"USA, UK",Richard Attenborough,"Drama, History, War","English, German, Dutch, Polish, Latin",N/A,"Operation Market Garden, September 1944: The A...",https://images-na.ssl-images-amazon.com/images...,PG,...,True,175 min,A Bridge Too Far,movie,"Cornelius Ryan (book), William Goldman (screen...",1977,tt0075784,7.4,"41,825",NaN
9,"Madeline Carroll, Callan McAuliffe, Rebecca De...",2 wins & 2 nominations.,USA,Rob Reiner,"Comedy, Drama, Romance",English,45,Two eighth-graders start to have feelings for ...,https://images-na.ssl-images-amazon.com/images...,PG,...,True,90 min,Flipped,movie,"Rob Reiner (screenplay), Andrew Scheinman (scr...",2010,tt0817177,7.7,"64,114",NaN


We see that we have included some TV series as well. We need to remove these and scrape additional samples

In [150]:
dftomerge['Type'].value_counts()

movie     243
series      7
Name: Type, dtype: int64

In [153]:
dftomerge = dftomerge.loc[dftomerge['Type'] == 'movie']
dftomerge.shape

(243, 21)

In [154]:
moresample = random.sample(samplelis, 7)

In [158]:
dftomergelis = [str(x) for x in dftomerge['imdbID']]

In [160]:
movie_rows = []
for movie in moresample:
    r = requests.get('http://www.omdbapi.com/?i=%s&r=json' % movie)
    json_content = json.loads(r.content)
    movie_rows.append(json_content)
    time.sleep(2)

In [161]:
dfsample3 = pd.DataFrame(movie_rows)
dfsample3.to_csv('dfsample3.csv', encoding = 'utf-8')

In [162]:
dfsample3

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,Released,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes
0,"Zac Efron, Wes Bentley, Emily Ratajkowski, Jon...",N/A,"UK, France, USA",Max Joseph,"Drama, Music, Romance",English,46,Caught between a forbidden romance and the exp...,https://images-na.ssl-images-amazon.com/images...,R,28 Aug 2015,True,96 min,We Are Your Friends,movie,"Max Joseph (screenplay), Meaghan Oppenheimer (...",2015,tt3787590,6.1,"24,225"
1,"James Woods, Sonja Smits, Debbie Harry, Peter ...",3 wins & 7 nominations.,Canada,David Cronenberg,"Horror, Sci-Fi, Thriller",English,60,A sleazy cable-TV programmer begins to see his...,https://images-na.ssl-images-amazon.com/images...,R,04 Feb 1983,True,87 min,Videodrome,movie,David Cronenberg,1983,tt0086541,7.3,"63,782"
2,"Robert Downey Jr., Gwyneth Paltrow, Don Cheadl...",Nominated for 1 Oscar. Another 17 wins & 55 no...,"China, USA",Shane Black,"Action, Adventure, Sci-Fi",English,62,When Tony Stark's world is torn apart by a for...,https://images-na.ssl-images-amazon.com/images...,PG-13,03 May 2013,True,130 min,Iron Man 3,movie,"Drew Pearce (screenplay), Shane Black (screenp...",2013,tt1300854,7.2,"583,743"
3,"Elizabeth Berkley, Kyle MacLachlan, Gina Gersh...",10 wins & 10 nominations.,"France, USA",Paul Verhoeven,Drama,English,16,"Nomi, a young drifter, arrives in Las Vegas to...",https://images-na.ssl-images-amazon.com/images...,NC-17,22 Sep 1995,True,128 min,Showgirls,movie,Joe Eszterhas,1995,tt0114436,4.6,"51,685"
4,"Kirk Cameron, Darren Doane, Bridgette Cameron,...",4 wins & 2 nominations.,USA,Darren Doane,"Comedy, Family",English,18,Kirk is enjoying the annual Christmas party ex...,https://images-na.ssl-images-amazon.com/images...,PG,14 Nov 2014,True,80 min,Saving Christmas,movie,"Darren Doane, Cheston Hervey",2014,tt4009460,1.6,"12,497"
5,"Anika Noni Rose, Bruno Campos, Keith David, Mi...",Nominated for 3 Oscars. Another 12 wins & 39 n...,USA,"Ron Clements, John Musker","Animation, Adventure, Comedy","English, French",73,"A waitress, desperate to fulfill her dreams as...",https://images-na.ssl-images-amazon.com/images...,G,11 Dec 2009,True,97 min,The Princess and the Frog,movie,"Ron Clements (story by), John Musker (story by...",2009,tt0780521,7.1,"93,672"
6,"Frank Grillo, Carmen Ejogo, Zach Gilford, Kiel...",2 wins & 6 nominations.,"France, USA",James DeMonaco,"Action, Horror, Sci-Fi",English,50,Three groups of people intertwine and are left...,https://images-na.ssl-images-amazon.com/images...,R,18 Jul 2014,True,103 min,The Purge: Anarchy,movie,"James DeMonaco, James DeMonaco (characters cre...",2014,tt2975578,6.5,"104,795"


In [163]:
listofdf = [dftomerge, dfsample3]
dftomerge = pd.concat(listofdf)
dftomerge.reset_index(drop = True)

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,...,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes,totalSeasons
0,"Adam Beach, Evan Adams, Irene Bedard, Gary Farmer",11 wins & 8 nominations.,"Canada, USA",Chris Eyre,"Comedy, Drama",English,76,Young Indian man Thomas is a nerd in his reser...,https://images-na.ssl-images-amazon.com/images...,PG-13,...,True,89 min,Smoke Signals,movie,"Sherman Alexie (book), Sherman Alexie (screenp...",1998,tt0120321,7.2,"8,416",NaN
1,"Robin Williams, Mandy Moore, John Krasinski, E...",1 nomination.,"USA, Australia",Ken Kwapis,"Comedy, Romance",English,25,A reverend puts an engaged couple through a gr...,https://images-na.ssl-images-amazon.com/images...,PG-13,...,True,91 min,License to Wed,movie,"Kim Barker (screenplay), Tim Rasmussen (screen...",2007,tt0762114,5.3,"33,017",NaN
2,"Victor Rasuk, Donna Maldonado, Kevin Rivera, K...",4 wins & 12 nominations.,"France, USA",Peter Sollett,"Drama, Romance","English, Spanish",83,A Lower East Side teen-ager struggles to find ...,https://images-na.ssl-images-amazon.com/images...,R,...,True,88 min,Raising Victor Vargas,movie,"Peter Sollett (story), Peter Sollett, Eva Vive...",2002,tt0316188,7.2,"4,811",NaN
3,"Russell Crowe, Elizabeth Banks, Michael Buie, ...",4 nominations.,"USA, France",Paul Haggis,"Crime, Drama, Romance",English,52,A married couple's life is turned upside down ...,https://images-na.ssl-images-amazon.com/images...,PG-13,...,True,133 min,The Next Three Days,movie,"Paul Haggis (screenplay), Fred Cavayé (screenp...",2010,tt1458175,7.4,"151,013",NaN
4,"Dane Cook, Kate Hudson, Alec Baldwin, Jason Biggs",1 nomination.,USA,Howard Deutch,"Comedy, Romance",English,34,Tank faces the ultimate test of friendship whe...,https://images-na.ssl-images-amazon.com/images...,R,...,True,101 min,My Best Friend's Girl,movie,Jordan Cahan,2008,tt1046163,5.9,"37,990",NaN
5,"Mike Starr, Paul Herman, Don Pardo, Martin Ros...",Nominated for 2 Oscars. Another 3 wins & 7 nom...,USA,Woody Allen,Comedy,"English, Spanish, Italian",N/A,A nostalgic look at radio's golden age focusin...,https://images-na.ssl-images-amazon.com/images...,PG,...,True,88 min,Radio Days,movie,Woody Allen,1987,tt0093818,7.6,"25,475",NaN
6,"Nicole Kidman, Joseph Fiennes, Hugo Weaving, L...",1 win & 6 nominations.,"Ireland, Australia",Kim Farrant,"Drama, Mystery, Thriller",English,42,A family finds their dull life in a rural outb...,https://images-na.ssl-images-amazon.com/images...,R,...,True,112 min,Strangerland,movie,"Michael Kinirons, Fiona Seres (story), Fiona S...",2015,tt2325977,5.2,"6,298",NaN
7,"Jordana Brewster, Taylor Handley, Diora Baird,...",5 nominations.,USA,Jonathan Liebesman,Horror,English,30,On one last road trip before they're sent to s...,https://images-na.ssl-images-amazon.com/images...,R,...,True,91 min,The Texas Chainsaw Massacre: The Beginning,movie,"Sheldon Turner (screenplay), Sheldon Turner (s...",2006,tt0420294,5.9,"57,990",NaN
8,"Siem Vroom, Marlies van Alcmaer, Erik van 't W...",Won 3 BAFTA Film Awards. Another 4 wins & 4 no...,"USA, UK",Richard Attenborough,"Drama, History, War","English, German, Dutch, Polish, Latin",N/A,"Operation Market Garden, September 1944: The A...",https://images-na.ssl-images-amazon.com/images...,PG,...,True,175 min,A Bridge Too Far,movie,"Cornelius Ryan (book), William Goldman (screen...",1977,tt0075784,7.4,"41,825",NaN
9,"Madeline Carroll, Callan McAuliffe, Rebecca De...",2 wins & 2 nominations.,USA,Rob Reiner,"Comedy, Drama, Romance",English,45,Two eighth-graders start to have feelings for ...,https://images-na.ssl-images-amazon.com/images...,PG,...,True,90 min,Flipped,movie,"Rob Reiner (screenplay), Andrew Scheinman (scr...",2010,tt0817177,7.7,"64,114",NaN


In [167]:
dftomerge.drop('totalSeasons', axis = 1, inplace = True)

In [169]:
dftomerge['top250'] = 0

In [183]:
df.columns

Index([u'Actors', u'Awards', u'Country', u'Director', u'Genre', u'Language',
       u'Metascore', u'Plot', u'Poster', u'Rated', u'Released', u'Response',
       u'Runtime', u'Title', u'Type', u'Writer', u'Year', u'imdbID',
       u'imdbRating', u'imdbVotes', u'top250'],
      dtype='object')

In [184]:
dftomerge.columns

Index([    u'Actors',     u'Awards',    u'Country',   u'Director',
            u'Genre',   u'Language',  u'Metascore',       u'Plot',
           u'Poster',      u'Rated',   u'Released',   u'Response',
          u'Runtime',      u'Title',       u'Type',     u'Writer',
             u'Year',     u'imdbID', u'imdbRating',  u'imdbVotes',
           u'top250'],
      dtype='object')

In [185]:
dffinal = pd.concat([df,dftomerge])
dffinal.shape

(500, 21)

In [188]:
dffinal.reset_index(drop = True)

,Actors,Awards,Country,Director,Genre,Language,Metascore,Plot,Poster,Rated,...,Response,Runtime,Title,Type,Writer,Year,imdbID,imdbRating,imdbVotes,top250
0,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Nominated for 7 Oscars. Another 19 wins & 30 n...,USA,Frank Darabont,"Crime, Drama",English,80,Two imprisoned men bond over a number of years...,https://images-na.ssl-images-amazon.com/images...,R,...,True,142 min,The Shawshank Redemption,movie,"Stephen King (short story ""Rita Hayworth and S...",1994,tt0111161,9.3,"1,786,262",1
1,"Marlon Brando, Al Pacino, James Caan, Richard ...",Won 3 Oscars. Another 23 wins & 27 nominations.,USA,Francis Ford Coppola,"Crime, Drama","English, Italian, Latin",100,The aging patriarch of an organized crime dyna...,https://images-na.ssl-images-amazon.com/images...,R,...,True,175 min,The Godfather,movie,"Mario Puzo (screenplay), Francis Ford Coppola ...",1972,tt0068646,9.2,"1,219,320",1
2,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",Won 6 Oscars. Another 10 wins & 20 nominations.,USA,Francis Ford Coppola,"Crime, Drama","English, Italian, Spanish, Latin, Sicilian",80,The early life and career of Vito Corleone in ...,https://images-na.ssl-images-amazon.com/images...,R,...,True,202 min,The Godfather: Part II,movie,"Francis Ford Coppola (screenplay), Mario Puzo ...",1974,tt0071562,9,"839,135",1
3,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",Won 2 Oscars. Another 146 wins & 142 nominations.,"USA, UK",Christopher Nolan,"Action, Crime, Drama","English, Mandarin",82,When the menace known as the Joker wreaks havo...,https://images-na.ssl-images-amazon.com/images...,PG-13,...,True,152 min,The Dark Knight,movie,"Jonathan Nolan (screenplay), Christopher Nolan...",2008,tt0468569,9,"1,754,213",1
4,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",Nominated for 3 Oscars. Another 16 wins & 8 no...,USA,Sidney Lumet,"Crime, Drama",English,NaN,A jury holdout attempts to prevent a miscarria...,https://images-na.ssl-images-amazon.com/images...,APPROVED,...,True,96 min,12 Angry Men,movie,"Reginald Rose (story), Reginald Rose (screenplay)",1957,tt0050083,8.9,"481,606",1
5,"Liam Neeson, Ben Kingsley, Ralph Fiennes, Caro...",Won 7 Oscars. Another 71 wins & 33 nominations.,USA,Steven Spielberg,"Biography, Drama, History","English, Hebrew, German, Polish",93,"In German-occupied Poland during World War II,...",https://images-na.ssl-images-amazon.com/images...,R,...,True,195 min,Schindler's List,movie,"Thomas Keneally (book), Steven Zaillian (scree...",1993,tt0108052,8.9,"909,242",1
6,"Tim Roth, Amanda Plummer, Laura Lovelace, John...",Won 1 Oscar. Another 60 wins & 65 nominations.,USA,Quentin Tarantino,"Crime, Drama","English, Spanish, French",94,"The lives of two mob hit men, a boxer, a gangs...",https://images-na.ssl-images-amazon.com/images...,R,...,True,154 min,Pulp Fiction,movie,"Quentin Tarantino (story), Roger Avary (story)...",1994,tt0110912,8.9,"1,388,612",1
7,"Noel Appleby, Ali Astin, Sean Astin, David Aston",Won 11 Oscars. Another 190 wins & 119 nominati...,"USA, New Zealand",Peter Jackson,"Adventure, Drama, Fantasy","English, Quenya, Old English, Sindarin",94,Gandalf and Aragorn lead the World of Men agai...,https://images-na.ssl-images-amazon.com/images...,PG-13,...,True,201 min,The Lord of the Rings: The Return of the King,movie,"J.R.R. Tolkien (novel), Fran Walsh (screenplay...",2003,tt0167260,8.9,"1,280,900",1
8,"Eli Wallach, Clint Eastwood, Lee Van Cleef, Al...",1 win & 2 nominations.,"Italy, Spain, West Germany, USA",Sergio Leone,Western,Italian,90,A bounty hunting scam joins two men in an unea...,https://images-na.ssl-images-amazon.com/images...,APPROVED,...,True,161 min,"The Good, the Bad and the Ugly",movie,"Luciano Vincenzoni (story), Sergio Leone (stor...",1966,tt0060196,8.9,"528,446",1
9,"Edward Norton, Brad Pitt, Meat Loaf, Zach Grenier",Nominated for 1 Oscar. Another 10 wins & 31 no...,"USA, Germany",David Fincher,Drama,English,66,"An insomniac office worker, looking for a way .

In [189]:
dffinal.to_csv('dffinal.csv', encoding = 'utf-8')